In [6]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import h5py
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score

# label_dict = {'good':0, 'holes_cuts':1, 'threaderror':2, 'oilstains_colorerror':3, 'wrinkles':4, 'foreignbodies':5}
file_path = r"C:\Users\LENOVO\CV\Automated-Fabric-Defect-Inspection\textile_defect_data.hdF5"
imgs = h5py.File(file_path)['jute_defect_imgs'][:]
labels = h5py.File(file_path)['jute_defect_labels'][:]

# Define the displacement vector (1, 0) for horizontal pairs
displacement = (1, 0)

# Define the number of histogram bins
num_bins = 256

# Define the feature vector
feature_vector = []

for img in imgs:

    # Calculate the co-occurrence matrix using OpenCV

    co_matrix = cv2.calcHist([img, img], [0, 1], None, [num_bins, num_bins], [0, num_bins, 0, num_bins], accumulate=False)

    # Normalize the co-occurrence matrix
    co_matrix_normalized = cv2.normalize(co_matrix, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    # Calculate contrast
    contrast = np.sum((co_matrix_normalized * np.arange(num_bins)[:, np.newaxis] - np.mean(co_matrix_normalized)) ** 2)

    # Calculate energy
    energy = np.sum(co_matrix_normalized ** 2)

    # Calculate homogeneity
    homogeneity = np.sum(co_matrix_normalized / (1 + np.abs(np.arange(num_bins)[:, np.newaxis] - np.arange(num_bins))))

    # Calculate correlation
    x, y = np.meshgrid(np.arange(num_bins), np.arange(num_bins))
    correlation = np.sum((co_matrix_normalized * (x - np.mean(x)) * (y - np.mean(y))) / (np.std(x) * np.std(y)))

    # Add the feature vector for the current image to the feature vector list
    feature_vector.append([contrast, energy, homogeneity, correlation])

# Convert the feature vector list to a numpy array
feature_vector = np.array(feature_vector)

# print(feature_vector)

# Print the feature vector for the first image
print(len(feature_vector))

# Principle components
pca=PCA()
principle_components=pca.fit_transform(feature_vector)

# svm
x_train,x_test,y_train,y_test=train_test_split(principle_components,labels,test_size=0.25,random_state=42)
classifier=SVC(decision_function_shape='ovo',kernel='rbf',random_state=42)
classifier.fit(x_train,y_train)

y_pred=classifier.predict(x_test)

print("Accuracy Score",accuracy_score(y_test,y_pred))


300
Accuracy Score 0.4666666666666667


# GLCM

In [2]:
import numpy as np
from skimage.feature import graycomatrix
from skimage import data

from skimage.feature import graycoprops

# Assuming you have a co-occurrence matrix stored in 'co_occurrence_matrix'
import numpy as np
import cv2 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import h5py
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score

# label_dict = {'good':0, 'holes_cuts':1, 'threaderror':2, 'oilstains_colorerror':3, 'wrinkles':4, 'foreignbodies':5}
file_path = r"C:\Users\LENOVO\CV\Automated-Fabric-Defect-Inspection\textile_defect_data.hdF5"
imgs = h5py.File(file_path)['jute_defect_imgs'][:]
labels = h5py.File(file_path)['jute_defect_labels'][:]

# Define the feature vector
feature_vector = []

# Step 1: Load the image 

for image_gray in imgs:
    


    # Step 2: Choose the direction for the co-occurrence matrix
    direction = (1, 0)  # Horizontal direction

    # Step 3: Define the distance and offset for the co-occurrence matrix
    distance = 1
    offset = (0, 1)

    # Step 4: Compute the co-occurrence matrix
    co_occurrence_matrix = graycomatrix(image_gray, distances=[distance], angles=[np.arctan2(*direction)], levels=256, symmetric=True, normed=True)

    # Step 5: Access the computed co-occurrence matrix
    matrix = co_occurrence_matrix[:, :, 0, 0]  # Extract the co-occurrence matrix from the result



    # Calculate contrast
    contrast = graycoprops(co_occurrence_matrix, 'contrast').flatten()

   
    # Calculate correlation
    correlation = graycoprops(co_occurrence_matrix, 'correlation').flatten()

    # Calculate energy
    energy = graycoprops(co_occurrence_matrix, 'energy').flatten()

    # Calculate homogeneity
    homogeneity = graycoprops(co_occurrence_matrix, 'homogeneity').flatten()

    # Concatenate the statistical measures into a feature vector
    feature_vector.append([contrast, energy, homogeneity, correlation])
    
feature_vector =np.array(feature_vector)
# print(feature_vector[0])


features=[]
for i in feature_vector:
    feature=[]
#     print(i)
    for j in i:
        feature.append(j[0])
    features.append(feature)    
feature=np.array(feature)       
# print(features)
        
# print(type(contrast))

# Print the feature vector for the first image
# print(len(features))


# svm
x_train,x_test,y_train,y_test=train_test_split(features,labels,test_size=0.3,random_state=42)
classifier=SVC(decision_function_shape='ovo',kernel='linear',random_state=42)
classifier.fit(x_train,y_train)

y_pred=classifier.predict(x_test)

print("Accuracy Score",accuracy_score(y_test,y_pred))





Accuracy Score 0.24444444444444444


In [3]:
[i[0] for i in feature_vector[0]]

[239.5434911121983,
 0.021540677649311248,
 0.08288298577691895,
 0.8750597547060773]

In [4]:
for i in feature_vector[0]:
    print(i[0])

239.5434911121983
0.021540677649311248
0.08288298577691895
0.8750597547060773
